# SOTIF Uncertainty Evaluation for LiDAR Object Detection

**Uncertainty Evaluation to Support Safety of the Intended Functionality Analysis for Identifying Performance Insufficiencies in ML-Based LiDAR Object Detection**

*Milin Patel and Rolf Jung, Kempten University of Applied Sciences*

---

## What This Notebook Does

This notebook implements the complete **five-stage evaluation methodology** from the paper and produces all **four SOTIF artefacts** mapped to ISO 21448 clauses:

| Stage | Description | Section |
|-------|-------------|---------|
| Stage 1 | Ensemble Inference (K=6 SECOND detectors) | 3.2 |
| Stage 2 | Cross-Member Association & Uncertainty Indicators | 3.3 |
| Stage 3 | Correctness Determination (greedy BEV IoU matching) | 3.4 |
| Stage 4 | Metric Computation (discrimination, calibration, operating characteristics) | 3.5 |
| Stage 5 | SOTIF Analysis (TC ranking, frame flags, acceptance gates) | 3.6 |

| SOTIF Artefact | ISO 21448 Clause | What It Produces |
|----------------|------------------|------------------|
| Per-detection discrimination | Clause 7 | AUROC separating TP from FP |
| Triggering condition ranking | Clause 7 | Conditions ranked by FP share and uncertainty |
| Frame-level triage | Clause 7 | Flagged frames for investigation (Area 3 to Area 2) |
| Acceptance criteria | Clause 11 | Operating points with coverage and FAR |

**No GPU required.** Uses synthetic data matching the paper's statistics. For real data, see Section 10.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection/blob/main/notebooks/SOTIF_Uncertainty_Evaluation_Demo.ipynb)

## 0. Setup

In [ ]:
import os, sys
if 'google.colab' in str(get_ipython()):
    if not os.path.exists('SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection'):
        !git clone https://github.com/milinpatel07/SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection.git
    os.chdir('SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection')
    !pip install -q -e .
else:
    sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt
%matplotlib inline

print(f'NumPy {np.__version__} | Matplotlib {matplotlib.__version__}')
print('Setup complete.')

## 1. Generate Ensemble Data (Stage 1)

The demo generates synthetic ensemble outputs matching the paper's case study:
- **465 proposals**: 135 TP, 330 FP (BEV IoU >= 0.5)
- **K=6** SECOND detectors trained with different random seeds
- **22 environmental configurations** across 101 frames
- **4 triggering condition categories**: heavy rain, night, fog/visibility, other

In [ ]:
from sotif_uncertainty.demo_data import generate_demo_dataset, get_individual_member_aps

data = generate_demo_dataset(n_tp=135, n_fp=330, K=6, n_frames=101, seed=42)

scores = data['scores']           # (465, 6)
boxes = data['boxes']             # (465, 6, 7)
labels = data['labels']           # (465,)
frame_ids = data['frame_ids']     # (465,)
conditions = data['conditions']   # (465,)

print(f'Dataset Statistics (Section 5.1):')
print(f'  Total proposals:  {len(labels)}')
print(f'  True positives:   {np.sum(labels==1)}')
print(f'  False positives:  {np.sum(labels==0)}')
print(f'  FP:TP ratio:      {np.sum(labels==0)/np.sum(labels==1):.1f}:1')
print(f'  Ensemble size:    K={data["K"]}')
print(f'  Frames:           {data["n_frames"]}')
print(f'  Conditions:       {sorted(np.unique(conditions))}')

print(f'\nIndividual Member BEV AP (Table 1):')
for name, ap in get_individual_member_aps().items():
    print(f'  Member {name}: {ap:.2f}%')
print(f'  Range: {min(get_individual_member_aps().values()):.2f}% - {max(get_individual_member_aps().values()):.2f}%')

## 2. Stage 2: Uncertainty Indicators

Three uncertainty indicators per proposal, each capturing a distinct aspect of detection unreliability:

**Mean confidence** $\bar{s}_j = \frac{1}{K} \sum_{k=1}^{K} s_j^{(k)}$ -- existence uncertainty (Eq. 1)

**Confidence variance** $\sigma^2_{s,j} = \frac{1}{K-1} \sum_{k=1}^{K} (s_j^{(k)} - \bar{s}_j)^2$ -- epistemic uncertainty (Eq. 2)

**Geometric disagreement** $d_{\text{iou},j} = 1 - \frac{2}{K(K-1)} \sum_{u<v} \text{IoU}_{\text{BEV}}(b_j^{(u)}, b_j^{(v)})$ -- localisation uncertainty (Eq. 3)

In [ ]:
from sotif_uncertainty.uncertainty import compute_all_indicators

indicators = compute_all_indicators(scores, boxes)
mean_conf = indicators['mean_confidence']
conf_var = indicators['confidence_variance']
geo_disagree = indicators['geometric_disagreement']

tp = labels == 1
fp = labels == 0

print('Uncertainty Indicator Summary')
print('=' * 65)
print(f'{"Indicator":<25} {"TP mean":>10} {"FP mean":>10} {"TP 80th":>10} {"FP 80th":>10}')
print('-' * 65)
for name, vals in [('Mean confidence', mean_conf), ('Confidence variance', conf_var), ('Geo. disagreement', geo_disagree)]:
    print(f'{name:<25} {np.mean(vals[tp]):>10.4f} {np.mean(vals[fp]):>10.4f} '
          f'{np.percentile(vals[tp], 80):>10.5f} {np.percentile(vals[fp], 80):>10.5f}')

### 2.1 Indicator Distribution Analysis

In [ ]:
from sotif_uncertainty.visualization import plot_indicator_distributions
fig = plot_indicator_distributions(mean_conf, conf_var, geo_disagree, labels)
plt.show()

### 2.2 Uncertainty Landscape (Figure 7)

In [ ]:
from sotif_uncertainty.visualization import plot_scatter_confidence_variance
fig = plot_scatter_confidence_variance(mean_conf, conf_var, labels)
plt.show()

### 2.3 Ensemble Member Agreement

In [ ]:
from sotif_uncertainty.visualization import plot_member_agreement
fig = plot_member_agreement(scores, labels)
plt.show()

## 3. Stage 4: Discrimination Metrics (Table 3)

**Key question**: Does uncertainty identify which detections are incorrect?

AUROC measures ranking quality: 1.0 = perfect separation, 0.5 = random.

In [ ]:
from sotif_uncertainty.metrics import compute_all_metrics, compute_auroc_with_curve

metrics = compute_all_metrics(mean_conf, conf_var, geo_disagree, labels)
disc = metrics['discrimination']

print('Discrimination: AUROC for TP vs. FP Separation (Table 3)')
print('=' * 55)
print(f'  Mean Confidence (s_bar):          AUROC = {disc["auroc_mean_confidence"]:.3f}')
print(f'  Confidence Variance (sigma^2_s):   AUROC = {disc["auroc_confidence_variance"]:.3f}')
print(f'  Geometric Disagreement (d_iou):    AUROC = {disc["auroc_geometric_disagreement"]:.3f}')

In [ ]:
from sotif_uncertainty.visualization import plot_roc_curves

auroc_conf, fpr_conf, tpr_conf = compute_auroc_with_curve(mean_conf, labels, True)
auroc_var, fpr_var, tpr_var = compute_auroc_with_curve(-conf_var, labels, True)
auroc_geo, fpr_geo, tpr_geo = compute_auroc_with_curve(-geo_disagree, labels, True)

fig = plot_roc_curves({
    'Mean Confidence': (fpr_conf, tpr_conf, auroc_conf),
    'Confidence Variance': (fpr_var, tpr_var, auroc_var),
    'Geometric Disagreement': (fpr_geo, tpr_geo, auroc_geo),
})
plt.show()

## 4. Stage 4: Calibration Metrics (Table 4)

**Key question**: Do confidence values match observed accuracy?

High discrimination (AUROC~1.0) and moderate calibration (ECE~0.2) coexist because they measure different properties. Acceptance gates depend on ranking, not absolute calibration (Section 6.2).

In [ ]:
cal = metrics['calibration']
rc = metrics['risk_coverage']

print('Calibration and Scoring Metrics (Table 4)')
print('=' * 40)
print(f'  ECE:         {cal["ece"]:.3f}')
print(f'  NLL:         {cal["nll"]:.3f}')
print(f'  Brier Score: {cal["brier"]:.3f}')
print(f'  AURC:        {rc["aurc"]:.3f}')

In [ ]:
from sotif_uncertainty.visualization import plot_reliability_diagram, plot_risk_coverage

fig = plot_reliability_diagram(cal['bin_accuracies'], cal['bin_confidences'], cal['bin_counts'], cal['ece'])
plt.show()

In [ ]:
fig = plot_risk_coverage(rc['coverages'], rc['risks'], rc['aurc'])
plt.show()

## 5. Stage 4: Operating Points / Acceptance Gates (Table 6)

The acceptance gate (Eq. 7) filters detections deterministically:

$$G(\bar{s}_j, \sigma^2_{s,j}, d_{\text{iou},j}) = [\bar{s}_j \geq \tau_s] \wedge [\sigma^2_{s,j} \leq \tau_v] \wedge [d_{\text{iou},j} \leq \tau_d]$$

ISO 21448 Clause 11 requires documented acceptance criteria. The operating point table provides this.

In [ ]:
from sotif_uncertainty.sotif_analysis import compute_operating_points, find_optimal_gate

conf_only = compute_operating_points(
    mean_conf, conf_var, geo_disagree, labels,
    tau_s_range=np.array([0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90]),
)
multi = compute_operating_points(
    mean_conf, conf_var, geo_disagree, labels,
    tau_s_range=np.array([0.60, 0.65, 0.70]),
    tau_v_range=np.array([0.002, 0.003]),
)
all_points = conf_only + multi

print('Operating Points Table (Table 6 / Clause 11)')
print('=' * 70)
print(f'{"Gate":<35} {"Cov.":>6} {"Ret.":>5} {"FP":>4} {"FAR":>8}')
print('-' * 70)
for p in all_points:
    if p['coverage'] > 0:
        print(f'{p["gate"]:<35} {p["coverage"]:>6.3f} {p["retained"]:>5d} {p["fp"]:>4d} {p["far"]:>8.3f}')

optimal = find_optimal_gate(
    mean_conf, conf_var, geo_disagree, labels, alpha=0.0,
    tau_s_range=np.arange(0.50, 0.91, 0.05),
    tau_v_range=np.array([np.inf, 0.002, 0.003]),
)
if optimal:
    print(f'\nOptimal zero-FAR gate: {optimal["gate"]}')
    print(f'  Coverage: {optimal["coverage"]:.3f} ({optimal["retained"]} detections, all correct)')

In [ ]:
from sotif_uncertainty.visualization import plot_operating_points_comparison, plot_operating_point_heatmap

fig = plot_operating_points_comparison(all_points)
plt.show()

In [ ]:
fig = plot_operating_point_heatmap(mean_conf, conf_var, geo_disagree, labels)
plt.show()

## 6. Stage 5: Triggering Condition Identification (Clause 7, Table 7)

Rank conditions by FP share and mean uncertainty. If both rankings agree, uncertainty analysis independently identifies the triggering conditions that error analysis identifies -- even without ground truth.

In [ ]:
from sotif_uncertainty.sotif_analysis import rank_triggering_conditions

tc_results = rank_triggering_conditions(conditions, labels, mean_conf, conf_var)

print('Triggering Condition Ranking (Table 7 / Clause 7)')
print('=' * 75)
print(f'{"Condition":<18} {"FP Share":>10} {"Mean s (FP)":>12} {"Mean var (FP)":>14} {"TP":>5} {"FP":>5}')
print('-' * 75)
for tc in tc_results:
    print(f'{tc["condition"]:<18} {tc["fp_share"]:>10.1%} {tc["mean_conf_fp"]:>12.3f} '
          f'{tc["mean_var_fp"]:>14.5f} {tc["tp_count"]:>5d} {tc["fp_count"]:>5d}')

# Verify ranking agreement
by_share = [tc['condition'] for tc in tc_results]
by_var = [tc['condition'] for tc in sorted(tc_results, key=lambda x: x['mean_var_fp'], reverse=True)]
print(f'\nRanking by FP share:    {by_share}')
print(f'Ranking by mean FP var: {by_var}')
print(f'Rankings match: {by_share == by_var}')

In [ ]:
from sotif_uncertainty.visualization import plot_tc_ranking, plot_condition_breakdown, plot_condition_boxplots

fig = plot_tc_ranking(tc_results)
plt.show()

In [ ]:
fig = plot_condition_breakdown(labels, conditions)
plt.show()

In [ ]:
fig = plot_condition_boxplots(mean_conf, conf_var, labels, conditions)
plt.show()

## 7. Stage 5: Frame-Level Triage (Clause 7)

Flag frames containing high-uncertainty false positives for targeted investigation.
Supports the Area 3 -> Area 2 transition in ISO 21448.

In [ ]:
from sotif_uncertainty.sotif_analysis import flag_frames, compute_frame_summary
from sotif_uncertainty.visualization import plot_frame_risk_scatter

flag_results = flag_frames(frame_ids, labels, conf_var, percentile=80.0)
frame_summaries = compute_frame_summary(frame_ids, labels, mean_conf, conf_var, conditions)

print('Frame-Level Triage (Clause 7)')
print('=' * 50)
print(f'  Variance threshold (80th pct FP): {flag_results["threshold"]:.5f}')
print(f'  Total frames:  {flag_results["total_frames"]}')
print(f'  Flagged frames: {flag_results["flagged_count"]}')

fig = plot_frame_risk_scatter(frame_summaries)
plt.show()

## 8. ISO 21448 Scenario Categorisation

In [ ]:
from sotif_uncertainty.visualization import plot_iso21448_scenario_grid
fig = plot_iso21448_scenario_grid()
plt.show()

## 9. Summary Dashboard

In [ ]:
from sotif_uncertainty.visualization import plot_summary_dashboard
fig = plot_summary_dashboard(metrics, mean_conf, conf_var, labels, tc_results)
plt.show()

## 10. Complete SOTIF Artefact Summary (Table 5)

In [ ]:
print('SOTIF Artefact Summary (Table 5)')
print('=' * 80)

artefacts = [
    ('Per-detection AUROC, AURC',
     'Performance insufficiency identification (Clause 7)',
     f'AUROC(conf)={disc["auroc_mean_confidence"]:.3f}, AUROC(var)={disc["auroc_confidence_variance"]:.3f}'),
    ('Per-condition FP rate & uncertainty ranking',
     'Triggering condition identification (Clause 7)',
     f'{tc_results[0]["condition"]} ({tc_results[0]["fp_share"]:.0%}), '
     f'{tc_results[1]["condition"]} ({tc_results[1]["fp_share"]:.0%})'),
    ('Frame flags (high-variance FP)',
     'Scenario triage: Area 3 -> Area 2 (Clause 7)',
     f'{flag_results["flagged_count"]}/{flag_results["total_frames"]} frames flagged'),
    ('Coverage & FAR at thresholds',
     'Acceptance criteria input (Clause 11)',
     f'At s>=0.70: cov={[p for p in conf_only if abs(p["tau_s"]-0.70)<0.01][0]["coverage"]:.3f}, '
     f'FAR={[p for p in conf_only if abs(p["tau_s"]-0.70)<0.01][0]["far"]:.3f}' if any(abs(p['tau_s']-0.70)<0.01 for p in conf_only) else 'N/A'),
    ('ECE, NLL, Brier',
     'Confidence interpretation (Clause 10)',
     f'ECE={cal["ece"]:.3f}, NLL={cal["nll"]:.3f}, Brier={cal["brier"]:.3f}'),
]

for output, activity, value in artefacts:
    print(f'\n  Output:   {output}')
    print(f'  Clause:   {activity}')
    print(f'  Result:   {value}')

## 11. Save All Figures

In [ ]:
from sotif_uncertainty.visualization import generate_all_figures

figures = generate_all_figures(
    metrics=metrics,
    mean_conf=mean_conf,
    conf_var=conf_var,
    labels=labels,
    frame_summaries=frame_summaries,
    tc_results=tc_results,
    operating_points=all_points,
    output_dir='Analysis',
    scores=scores,
    geo_disagree=geo_disagree,
    conditions=conditions,
)

print(f'Generated {len(figures)} figures:')
for name in sorted(figures.keys()):
    print(f'  - Analysis/{name}.png')

## 12. Using Real Data

To run this evaluation on real ensemble outputs:

### Option A: KITTI Dataset
```bash
# 1. Download KITTI
python scripts/prepare_kitti.py --data_root data/kitti

# 2. Install OpenPCDet
git clone https://github.com/open-mmlab/OpenPCDet.git
cd OpenPCDet && pip install -e . && cd ..

# 3. Train 6 SECOND models with different seeds
bash scripts/train_ensemble.sh --seeds 0 1 2 3 4 5

# 4. Run ensemble inference
python scripts/run_inference.py --ckpt_dirs output/ensemble/seed_*

# 5. Evaluate
python scripts/evaluate.py --input results/ensemble_results.pkl
```

### Option B: CARLA Simulation (reproduces paper exactly)
```bash
# Generate KITTI-format data from CARLA with 22 weather configs
# See: https://github.com/fnozarian/CARLA-KITTI
```

### Available Datasets

| Dataset | Size | Format | Weather | Access |
|---------|------|--------|---------|--------|
| KITTI | ~29 GB | Native | Clear | Free (cvlibs.net) |
| nuScenes mini | ~4 GB | Convertible | Rain, night | Free (nuscenes.org) |
| MultiFog KITTI | Varies | KITTI | Synthetic fog | Free |
| CADC | ~7K frames | Custom | Snow/winter | Free (cadcd.uwaterloo.ca) |
| CARLA | Custom | KITTI | Any (simulated) | Free (carla.org) |